In [2]:
'''COPY PASTED CORE ON 01/02/2025'''
'''UPDATE AS NEEDED'''
import numpy as np
from numpy import random as npr
cooperate = np.array( [ 1, 0 ] )
defect = np.array( [ 0, 1 ] )
# defining all strategies
# # template function
# parameter_list = [ k, ]
# def strategy( parameter_list = [], round_number = 0, match_history = [[]], player_index = 0 ) : 
#     # move either cooperate or defect
#     move
#     return move 

# nice guy function
def nice_guy( parameter_list = [], round_number = 0, match_history = [[]], player_index = 0 ) : 
    move = cooperate
    return move 

# bad guy function
def bad_guy( parameter_list = [], round_number = 0, match_history = [[]], player_index = 0 ) : 
    move = defect
    return move 

# mainly nice guy function
def mainly_nice( parameter_list = [ 0.4 ], round_number = 0, match_history = [[]], player_index = 0 ) : 
    k = parameter_list[ 0 ]
    if k > 0.5 : 
        print( "Wrong function called: this is mainly_nice" )
        return - 1
    u = npr.random( 1 )
    if ( u < k ) : 
        move = defect
    else : 
        move = cooperate
    return move

# mainly bad guy function
def mainly_bad( parameter_list = [ 0.6 ], round_number = 0, match_history = [[]], player_index = 0 ) : 
    k = parameter_list[ 0 ]
    if k < 0.5 : 
        print( "Wrong function called: this is mainly_bad" )
        return - 1
    u = npr.random( 1 )
    if ( u < k ) : 
        move = defect
    else : 
        move = cooperate
    return move

# random guy: randomly defect or cooperate
def random_guy( parameter_list = [], round_number = 0, match_history = [[]], player_index = 0 ) :                                                                            # eccezione da gestire meglio?
    u = npr.random()                                                                           # corretto in random() invece di random(1)
    if ( u < 0.5 ) : move = defect
    else : move = cooperate
    return move

# tit_for_tat function
def tit_for_tat( parameter_list = [ ], round_number = 0, match_history = [[]], player_index = 0 ) : 
    if ( round_number == 0 ) : 
        move = cooperate
    else : 
        move = match_history[ round_number - 1 ][ 1 - player_index ]
    return move

# resentful guy function
def resentful_guy( parameter_list = [ ], round_number = 0, match_history = [[]], player_index = 0 ) : 
    move = cooperate
    for res in match_history :
        if np.all( res[ 1 - player_index  ] == defect ) :
            move = defect
    return move

# trusting guy (increasing odds of cooperating according to match history)
def trusting_guy( parameter_list = [ 0.5 ], round_number = 0, match_history = [[]], player_index = 0 ) :
    # starting trust
    k = parameter_list[ 0 ]
    # environmental trust
    if ( round_number > 0 ) :
        coop = 0. 
        for res in match_history : coop += res[ 1 - player_index  ][ 1 ]
        k = coop / len( match_history )
    u = npr.random( 1 )
    if ( u > k ) : 
        move = defect
    else : 
        move = cooperate
    return move

# balancing guy (cooperates only for an even cooperation history)
def balancing_guy( parameter_list = [], round_number = 0, match_history = [[]], player_index = 0 ) :
    move = defect
    if ( round_number > 0 ) :
        # total cooperation seen
        coop = 0
        for res in match_history : coop += ( res[ 0 ][ 1 ] + res[ 1 ][ 1 ])
        if ( ( coop % 2 ) == 0 ) : 
            move = cooperate
    return move

# Mid resentful: cooperates on the first move, and defects if the opponent has defected on any of the previous 3 moves, else cooperates
def mid_resentful( parameter_list = [], round_number = 0, match_history = [[]], player_index = 0):
    move = cooperate
    # checko gli ultimi 3 turni (o meno se sono all'inizio):
    start = max(0, round_number - 3)
    for round_i in range(start, round_number):
        # controllo se la mossa avversaria corrisponde ad un defect=[0,,1]
        if np.array_equal(match_history[round_i][1 - player_index], defect):   
            move = defect
            break  # non serve continuare il controllo, esco dal for
    return move

# reverse tit_for_tat function: start by defecting, then does the opposite of what the opponent has done in the previous move
def reverse_tit_for_tat( parameter_list = [], round_number = 0, match_history = [[]], player_index = 0 ) : 
    if ( round_number == 0 ) : move = defect
    else : move = [1,1] - match_history[ round_number - 1 ][ 1 - player_index ]  
            # in this way i obtain the opposite move: [1,1] - [1,0] = [0,1] and [1,1] - [0,1] = [1,0]
    return move


# dictionary of all strategies
strategies = {
    'NiceGuy' : nice_guy,
    'BadGuy' : bad_guy,
    'MainlyNice' : mainly_nice,
    'MainlyBad' : mainly_bad,
    'RandomGuy': random_guy,
    'TitForTat' : tit_for_tat,
    'ResentfulGuy' : resentful_guy,
    'TrustingGuy' : trusting_guy,
    'Thanos' : balancing_guy,
    'MidResentful': mid_resentful,
    'ReverseTft': reverse_tit_for_tat,
}

# default Payoff matrix
default_R = 2
default_S = 0
default_T = 3
default_P = 1
Payoff = np.array( [ [ default_R , default_S ], [ default_T, default_P ] ] )

# function to play one round 1 vs 1 for two different* strategies players
def round( key_1, key_2, parameter_list_1, parameter_list_2, round_number, match_history, M = Payoff ) :
    # getting strategies from dictionary
    S_1 = strategies[ key_1 ]
    S_2 = strategies[ key_2 ]
    # computing next move for each player
    u_1 = S_1( parameter_list_1, round_number, match_history, 0 )
    u_2 = S_2( parameter_list_2, round_number, match_history, 1 )
    # computing rewards for each player
    r_1 = np.dot( u_1, Payoff.dot(u_2))
    r_2 = np.dot( u_2, Payoff.dot(u_1))
    # updating match history
    match_history.append( [ u_1, u_2 ] )
    # returning results
    return r_1, r_2

# function to play a match of N rounds 1 vs 1 for two different* strategies player
def match( key_1, key_2, parameter_list_1, parameter_list_2, N_rounds = 10, M = Payoff ) : 
    # match history strarting empty
    history = [ ]
    # starting total rewards
    R_1 = 0
    R_2 = 0
    # list of partial sums
    Rewards = []
    for round_i in range( 0, N_rounds ) :
        r_1, r_2 = round( key_1, key_2, parameter_list_1, parameter_list_2, round_i, history, M )
        R_1 += r_1
        R_2 += r_2
        Rewards.append( [ R_1, R_2 ] )
    return Rewards


In [3]:
# Mattia
# I'll explore the ways to get the combinations of 2 players out of N partecipating in the tourney

# This default package initially seems nice: check documentantion here
# https://docs.python.org/3/library/itertools.html#itertools.combinations
import itertools

# We already got a problem: when using a dict, keys are unique, therefore if there are multiple players with the same type
# a dictionary of this type is not good
test_t = {
    'NiceGuy' : nice_guy,
    'BadGuy' : bad_guy,
    'MainlyNice' : mainly_nice,
    'BadGuy' : bad_guy,
    'BadGuy' : bad_guy,
    'NiceGuy' : nice_guy,
    }

# This only works if all the labels are different from each-other, which means only one player per type
print("If only 1 player per type is present, this works, howeverer it has poor generalization: \n")
a = itertools.combinations(strategies, 2)
for item in a: print(item)
print('\n')

# Example in the case of repeated keys
print("Repeated keys from dictionary do not work example: \n")
a = itertools.combinations(test_t, 2)
for item in a: print(item)
print('\n')

# This is not good: elements that appear only one time in the dict would go against themselves
print("Repeated keys using combination with replacement leads to having matches with oneself(see MainlyNice): \n")
a = itertools.combinations_with_replacement(test_t, 2)
for item in a: print(item)
print('\n')

# It seems the iterabole object is consumed after the cycle
print("Iterable obejct emptied after use: \n")
a_l = [x for x in a]
print(a_l)
print('\n')

# In order to avoid losing the object, a cast to a container is needed, e.g. a list
print("Proper way to use itertools: cast its output as a list in order not to lose it: \n")
a_list = list(itertools.combinations_with_replacement(test_t, 2))
print(a_list[0])
print(a_list[1])
print('\n')

# Original list is not modified
print("Original list is not modified: \n")
print(test_t)
print('\n')

# Therefore, this seems the best way to do it
print("Proper use example: \n")
test_list = ["NiceGuy","BadGuy","MainlyNice","BadGuy","BadGuy","NiceGuy"]
print(f"Starting from this list {test_list} \n")
b = list(itertools.combinations(test_list, 2))
print("Item print in for loop: \n")
for item in b: print(item)
print("\nWe can access the individual element with [0] for 'Player1' and [1] for 'Player2': \n")
for item in b: print(f"{item[0]}  {item[1]}")
print('\n')

If only 1 player per type is present, this works, howeverer it has poor generalization: 

('NiceGuy', 'BadGuy')
('NiceGuy', 'MainlyNice')
('NiceGuy', 'MainlyBad')
('NiceGuy', 'RandomGuy')
('NiceGuy', 'TitForTat')
('NiceGuy', 'ResentfulGuy')
('NiceGuy', 'TrustingGuy')
('NiceGuy', 'Thanos')
('NiceGuy', 'MidResentful')
('NiceGuy', 'ReverseTft')
('BadGuy', 'MainlyNice')
('BadGuy', 'MainlyBad')
('BadGuy', 'RandomGuy')
('BadGuy', 'TitForTat')
('BadGuy', 'ResentfulGuy')
('BadGuy', 'TrustingGuy')
('BadGuy', 'Thanos')
('BadGuy', 'MidResentful')
('BadGuy', 'ReverseTft')
('MainlyNice', 'MainlyBad')
('MainlyNice', 'RandomGuy')
('MainlyNice', 'TitForTat')
('MainlyNice', 'ResentfulGuy')
('MainlyNice', 'TrustingGuy')
('MainlyNice', 'Thanos')
('MainlyNice', 'MidResentful')
('MainlyNice', 'ReverseTft')
('MainlyBad', 'RandomGuy')
('MainlyBad', 'TitForTat')
('MainlyBad', 'ResentfulGuy')
('MainlyBad', 'TrustingGuy')
('MainlyBad', 'Thanos')
('MainlyBad', 'MidResentful')
('MainlyBad', 'ReverseTft')
('Random

In [4]:
# Check if the number of iterations is as expected by combination calculus
N = len(test_list)
k = 2
from scipy.special import comb
couples = int(comb(N,k))
obtained_couples = len(b)
print(f"Expected couples: {couples};   Obtained couples: {obtained_couples}")
# So the syntax is b[i][j] where i is the ordered number of the couple and j is the index for the player: 0 for player1, 1 for player2
print("Example b[0]: ", b[0])
print("Example b[0][0]: ", b[0][0])
print("Example b[0][1]: ", b[0][1])

Expected couples: 15;   Obtained couples: 15
Example b[0]:  ('NiceGuy', 'BadGuy')
Example b[0][0]:  NiceGuy
Example b[0][1]:  BadGuy


In [5]:
# Initialize from a list of frequencies
freq = np.arange(0,len(strategies),1)
player_participants = { x:i for x,i in zip(strategies.keys(),freq) }
player_list = []
print(player_participants, '\n')

for item in (list(player_participants.keys())):
    for i in range (player_participants[item]):
        player_list.append(item)
print(player_list, "\n")

# Initialize from a dictionary of frequencies
player_participants_dict = {
    'NiceGuy' : 0,
    'BadGuy' : 1,
    'MainlyNice' : 2,
    'MainlyBad' : 3,
    'TitForTat' : 4,
    'ResentfulGuy' : 5,
    'TrustingGuy' : 6,
    'Thanos' : 7,
}

player_list_from_dict = []
for item in (list(player_participants_dict.keys())):
    for i in range (player_participants_dict[item]):
        player_list_from_dict.append(item)
print(player_list_from_dict)

print("Are the two methods equivalent? ",player_list==player_list_from_dict)

{'NiceGuy': 0, 'BadGuy': 1, 'MainlyNice': 2, 'MainlyBad': 3, 'RandomGuy': 4, 'TitForTat': 5, 'ResentfulGuy': 6, 'TrustingGuy': 7, 'Thanos': 8, 'MidResentful': 9, 'ReverseTft': 10} 

['BadGuy', 'MainlyNice', 'MainlyNice', 'MainlyBad', 'MainlyBad', 'MainlyBad', 'RandomGuy', 'RandomGuy', 'RandomGuy', 'RandomGuy', 'TitForTat', 'TitForTat', 'TitForTat', 'TitForTat', 'TitForTat', 'ResentfulGuy', 'ResentfulGuy', 'ResentfulGuy', 'ResentfulGuy', 'ResentfulGuy', 'ResentfulGuy', 'TrustingGuy', 'TrustingGuy', 'TrustingGuy', 'TrustingGuy', 'TrustingGuy', 'TrustingGuy', 'TrustingGuy', 'Thanos', 'Thanos', 'Thanos', 'Thanos', 'Thanos', 'Thanos', 'Thanos', 'Thanos', 'MidResentful', 'MidResentful', 'MidResentful', 'MidResentful', 'MidResentful', 'MidResentful', 'MidResentful', 'MidResentful', 'MidResentful', 'ReverseTft', 'ReverseTft', 'ReverseTft', 'ReverseTft', 'ReverseTft', 'ReverseTft', 'ReverseTft', 'ReverseTft', 'ReverseTft', 'ReverseTft'] 

['BadGuy', 'MainlyNice', 'MainlyNice', 'MainlyBad', 'Mai

In [6]:
b = list(itertools.combinations(player_list_from_dict, 2))
print("Item print in for loop: \n")
for item in b: print(item)
# print("\nWe can access the individual element with [0] for 'Player1' and [1] for 'Player2': \n")
# for item in b: print(f"{item[0]}  {item[1]}")
# print('\n')

Item print in for loop: 

('BadGuy', 'MainlyNice')
('BadGuy', 'MainlyNice')
('BadGuy', 'MainlyBad')
('BadGuy', 'MainlyBad')
('BadGuy', 'MainlyBad')
('BadGuy', 'TitForTat')
('BadGuy', 'TitForTat')
('BadGuy', 'TitForTat')
('BadGuy', 'TitForTat')
('BadGuy', 'ResentfulGuy')
('BadGuy', 'ResentfulGuy')
('BadGuy', 'ResentfulGuy')
('BadGuy', 'ResentfulGuy')
('BadGuy', 'ResentfulGuy')
('BadGuy', 'TrustingGuy')
('BadGuy', 'TrustingGuy')
('BadGuy', 'TrustingGuy')
('BadGuy', 'TrustingGuy')
('BadGuy', 'TrustingGuy')
('BadGuy', 'TrustingGuy')
('BadGuy', 'Thanos')
('BadGuy', 'Thanos')
('BadGuy', 'Thanos')
('BadGuy', 'Thanos')
('BadGuy', 'Thanos')
('BadGuy', 'Thanos')
('BadGuy', 'Thanos')
('MainlyNice', 'MainlyNice')
('MainlyNice', 'MainlyBad')
('MainlyNice', 'MainlyBad')
('MainlyNice', 'MainlyBad')
('MainlyNice', 'TitForTat')
('MainlyNice', 'TitForTat')
('MainlyNice', 'TitForTat')
('MainlyNice', 'TitForTat')
('MainlyNice', 'ResentfulGuy')
('MainlyNice', 'ResentfulGuy')
('MainlyNice', 'ResentfulGuy')


In [7]:
# We use this dictionary as input to the tourney 
# Format is StrategyName: (Frequence, parameter_list)
player_participants_dict = {
    'NiceGuy' : (0,[]),
    'BadGuy' : (1,[]),
    'MainlyNice' : (2, [0.4] ),
    'MainlyBad' : (3, [0.6] ),
    'TitForTat' : (4,[]),
    'ResentfulGuy' : (5,[]),
    'TrustingGuy' : (6,[]),
    'Thanos' : (7,),
}

In [8]:
# Let's see how to navigate this dict:
keys = list(player_participants_dict.keys())
print("By using only player[key] we get a TUPLE, whose 1st element is the frequency, and the second a LIST of parameters")
print("For example for 'MainlyBad':  ",player_participants_dict[keys[3]] )
print(type(player_participants_dict[keys[3]]), "\n")

print("By using player[key][0] we get its 1st element, which is the frequency; this is an int")
print("For example for 'MainlyBad':  ",player_participants_dict[keys[3]][0] )
print(type(player_participants_dict[keys[3]][0]), "\n")

print("By using player[key][1] we get its 2nd element, which is the parameter list; this is a list")
print("For example for 'MainlyBad':  ",player_participants_dict[keys[3]][1] )
print(type(player_participants_dict[keys[3]][1]), "\n")

print("By using further indexes J we get the element in that position within the parameter list player[key][1][J] ")
print("For example for 'MainlyBad' with J=0:  ",player_participants_dict[keys[3]][1][0] )
print(type(player_participants_dict[keys[3]][1][0]), "\n")

By using only player[key] we get a TUPLE, whose 1st element is the frequency, and the second a LIST of parameters
For example for 'MainlyBad':   (3, [0.6])
<class 'tuple'> 

By using player[key][0] we get its 1st element, which is the frequency; this is an int
For example for 'MainlyBad':   3
<class 'int'> 

By using player[key][1] we get its 2nd element, which is the parameter list; this is a list
For example for 'MainlyBad':   [0.6]
<class 'list'> 

By using further indexes J we get the element in that position within the parameter list player[key][1][J] 
For example for 'MainlyBad' with J=0:   0.6
<class 'float'> 



In [9]:
# Now, since preserving player identity is a difficult task (that is, differentiating between two or more players with the same strategy)
# is difficult we'll be "cloning" all the combinations and add a numerical identifier in the name of the strategy to univocally identify the
# players. Example: if NiceGuy has a frequency of 3, we'll have the following: NiceGuy1, NiceGuy2, NiceGuy3.
# This also means we skip those with frequency 0 (those not participating in the tourney)
# By doing so we can compute the scores for each match and extrapolate whose score it was by matching the entries.

print("This is our starting list created from the above dictionary with frequencies: ")
player_list_from_dict = []
for item in (list(player_participants_dict.keys())):
    for i in range (player_participants_dict[item][0]):
        player_list_from_dict.append(item)
print(player_list_from_dict, "\n")

print("Let's count the entries for each strategy: ")
strat = list(player_participants_dict.keys())
freq = [player_participants_dict[key][0] for key in strat]
print("Strats: ",strat)
print("Freqs:  ",freq)
total = int(np.sum(freq))
print("The total is: ",total)
print("This means that the possibile combinations are: ", comb(total,2 ) )
named =[]
for key,f in zip (strat,freq):
    for j in range(1,f+1):
        if (f!=0):
            named.append(key + str(j))
print("Here's how the list gets converted with identity into the name: ")
print(named, "\n")

This is our starting list created from the above dictionary with frequencies: 
['BadGuy', 'MainlyNice', 'MainlyNice', 'MainlyBad', 'MainlyBad', 'MainlyBad', 'TitForTat', 'TitForTat', 'TitForTat', 'TitForTat', 'ResentfulGuy', 'ResentfulGuy', 'ResentfulGuy', 'ResentfulGuy', 'ResentfulGuy', 'TrustingGuy', 'TrustingGuy', 'TrustingGuy', 'TrustingGuy', 'TrustingGuy', 'TrustingGuy', 'Thanos', 'Thanos', 'Thanos', 'Thanos', 'Thanos', 'Thanos', 'Thanos'] 

Let's count the entries for each strategy: 
Strats:  ['NiceGuy', 'BadGuy', 'MainlyNice', 'MainlyBad', 'TitForTat', 'ResentfulGuy', 'TrustingGuy', 'Thanos']
Freqs:   [0, 1, 2, 3, 4, 5, 6, 7]
The total is:  28
This means that the possibile combinations are:  378.0
Here's how the list gets converted with identity into the name: 
['BadGuy1', 'MainlyNice1', 'MainlyNice2', 'MainlyBad1', 'MainlyBad2', 'MainlyBad3', 'TitForTat1', 'TitForTat2', 'TitForTat3', 'TitForTat4', 'ResentfulGuy1', 'ResentfulGuy2', 'ResentfulGuy3', 'ResentfulGuy4', 'ResentfulGuy

In [10]:
print("Check if the two match: ")
with_names = list(itertools.combinations(named,2))
b = list(itertools.combinations(player_list_from_dict, 2))
for i in range (len(b)): print( b[i], "     ", with_names[i]  )
print("The number of combinations obtained is: ", len(b))

Check if the two match: 
('BadGuy', 'MainlyNice')       ('BadGuy1', 'MainlyNice1')
('BadGuy', 'MainlyNice')       ('BadGuy1', 'MainlyNice2')
('BadGuy', 'MainlyBad')       ('BadGuy1', 'MainlyBad1')
('BadGuy', 'MainlyBad')       ('BadGuy1', 'MainlyBad2')
('BadGuy', 'MainlyBad')       ('BadGuy1', 'MainlyBad3')
('BadGuy', 'TitForTat')       ('BadGuy1', 'TitForTat1')
('BadGuy', 'TitForTat')       ('BadGuy1', 'TitForTat2')
('BadGuy', 'TitForTat')       ('BadGuy1', 'TitForTat3')
('BadGuy', 'TitForTat')       ('BadGuy1', 'TitForTat4')
('BadGuy', 'ResentfulGuy')       ('BadGuy1', 'ResentfulGuy1')
('BadGuy', 'ResentfulGuy')       ('BadGuy1', 'ResentfulGuy2')
('BadGuy', 'ResentfulGuy')       ('BadGuy1', 'ResentfulGuy3')
('BadGuy', 'ResentfulGuy')       ('BadGuy1', 'ResentfulGuy4')
('BadGuy', 'ResentfulGuy')       ('BadGuy1', 'ResentfulGuy5')
('BadGuy', 'TrustingGuy')       ('BadGuy1', 'TrustingGuy1')
('BadGuy', 'TrustingGuy')       ('BadGuy1', 'TrustingGuy2')
('BadGuy', 'TrustingGuy')       ('Bad

In [11]:
def tourney(participants={},N_gironi=1,N_rounds = 10, M = Payoff):
    # Retrieve only the names of the strategies repeated as many times as their frequency allows.
    player_list_from_dict = []
    for item in (list(participants.keys())):
        for i in range (participants[item][0]):
            player_list_from_dict.append(item)
    # Compute the combinations
    b = list(itertools.combinations(player_list_from_dict, 2))
    N_giornata = len(b)
    # List which will be used to have the scores of each match by the side of the two opponents
    list_with_scores = b
    # Loop over the combinations one by one
    for i in range (N_giornata):
        # Player1 and Player2 strategies
        player1 = b[i][0]
        player2 = b[i][1]
        # Player1 and Player2 parameters: [0] is the frequency, [1] is the list of parameters
        param1  = participants[player1][1]
        param2  = participants[player2][1]
        # Initiliaze an empty list to store the results over the match
        AllRewards = []
        # Do the match between the two players and update the rewards; The ordered syntax for match is:
        # match( key_1, key_2, parameter_list_1, parameter_list_2, N_rounds = 10, M = Payoff ) : 
        AllRewards =  match(player1, player2, param1, param2, N_rounds, M)
        # Retrieve the final scores for Player1 and Player2 as the last element in the list
        R1, R2 = AllRewards[-1][0], AllRewards[-1][1]
        list_with_scores[i] += (R1,R2)
    # Un-comment the following print to debug
    #for item in list_with_scores: print(item)
    
    # Good, now each combination has the corresponding score: we now need to assign it to the respective players and calculate the final score
    strat = list(participants.keys())
    freq = [participants[key][0] for key in strat]
    named =[]
    # Add identification to the layer trough the use of a numerical value from 1 up to their frequency BUT PRESERVE THE ORDER
    for key,f in zip (strat,freq):
        for j in range(1,f+1):
            if (f!=0):
                named.append(key + str(j))
    # Compute the permutations preserving the order
    with_names = list(itertools.combinations(named,2))
    # Copy the scores for each match over to the ones with identification
    # In list_with_scores[I][J] we have: I = ordered number of the combination, and J:
    # J=0 is 1st strategy; J=1 is 2nd strategy; J=2 is Player1 score; J=3 is Player2 score
    for i in range (N_giornata): with_names[i] += (list_with_scores[i][2],list_with_scores[i][3] )
    # Un-comment the following print to debug
    #for item in with_names: print(item)

    # Compute the score for each named player and add it to the list
    FinalScores = []
    for player in named:
        SumOver1 = np.sum([ with_names[i][2] for i in range (N_giornata) if (with_names[i][0] == player) ])
        SumOver2 = np.sum([ with_names[i][3] for i in range (N_giornata) if (with_names[i][1] == player) ])
        FinalScores.append(SumOver1 + SumOver2)
    Results = named
    
    for i in range (len(Results)):
        Results[i] = (Results[i],FinalScores[i])
    #Results += tuple( FinalScores )
    return ( Results )    

In [12]:
# Tourney test
test_part ={ 
    'NiceGuy' : (3,[]),
    'BadGuy' : (3,[]),
    'MainlyNice' : (4, [0.4] ),
    'MainlyBad' : (3, [0.6] ),
    'RandomGuy' : (4, []),
    'TitForTat' : (1,[]),
    'ResentfulGuy' : (2,[]),
    'TrustingGuy' : (1,[0.5]),
    'Thanos' : (2,[]),
    'MidResentful': (2,[]),
    'ReverseTft': (2,[]),
}

result = tourney(test_part,N_gironi=1,N_rounds = 10, M = Payoff)
for item in result: print(item)
print("\nHere's the Ranking at the end of the tourney:")
# Sorted gives a copy, while using LIST.sort() does it in place.
Ranking = sorted(result,reverse=True, key=lambda x: x[1])
for item in Ranking: print(item)

('NiceGuy1', 288.0)
('NiceGuy2', 286)
('NiceGuy3', 284)
('BadGuy1', 496)
('BadGuy2', 498)
('BadGuy3', 522)
('MainlyNice1', 319)
('MainlyNice2', 343)
('MainlyNice3', 369)
('MainlyNice4', 366)
('MainlyBad1', 406)
('MainlyBad2', 406)
('MainlyBad3', 401)
('RandomGuy1', 371)
('RandomGuy2', 397)
('RandomGuy3', 349)
('RandomGuy4', 381)
('TitForTat1', 386)
('ResentfulGuy1', 459)
('ResentfulGuy2', 434)
('TrustingGuy1', 316)
('Thanos1', 382)
('Thanos2', 412)
('MidResentful1', 422)
('MidResentful2', 430)
('ReverseTft1', 371)
('ReverseTft2', 359.0)

Here's the Ranking at the end of the tourney:
('BadGuy3', 522)
('BadGuy2', 498)
('BadGuy1', 496)
('ResentfulGuy1', 459)
('ResentfulGuy2', 434)
('MidResentful2', 430)
('MidResentful1', 422)
('Thanos2', 412)
('MainlyBad1', 406)
('MainlyBad2', 406)
('MainlyBad3', 401)
('RandomGuy2', 397)
('TitForTat1', 386)
('Thanos1', 382)
('RandomGuy4', 381)
('RandomGuy1', 371)
('ReverseTft1', 371)
('MainlyNice3', 369)
('MainlyNice4', 366)
('ReverseTft2', 359.0)
('Rando

In [13]:
print("Data format is as follows:")
print("First index is the default position in the list, Result[0]: ",result[0])
print("Second index is [0] for the strategy/player_name and [1] for its reward:")
print("Result[0][0]: ",result[0][0], "       Result[0][1]: " ,result[0][1])

Data format is as follows:
First index is the default position in the list, Result[0]:  ('NiceGuy1', 288.0)
Second index is [0] for the strategy/player_name and [1] for its reward:
Result[0][0]:  NiceGuy1        Result[0][1]:  288.0


In [14]:
labels = list(test_part.keys())
players = [ result[i][0] for i in range (len(result)) ]
rewards = [ result[i][1] for i in range (len(result)) ]
print(labels)
print(players)
mean_rewards= []
index = 0
for item in labels:
    # Make it a string to apply the count method: returns the number of times a substring is included in the main string
    f = str(players).count(item)
    # Compute the mean of their rewards
    mean = (np.sum(rewards[index:index+f]))/f
    mean_rewards.append(mean)
    index += f
for item in zip(labels,mean_rewards): print(item)

['NiceGuy', 'BadGuy', 'MainlyNice', 'MainlyBad', 'RandomGuy', 'TitForTat', 'ResentfulGuy', 'TrustingGuy', 'Thanos', 'MidResentful', 'ReverseTft']
['NiceGuy1', 'NiceGuy2', 'NiceGuy3', 'BadGuy1', 'BadGuy2', 'BadGuy3', 'MainlyNice1', 'MainlyNice2', 'MainlyNice3', 'MainlyNice4', 'MainlyBad1', 'MainlyBad2', 'MainlyBad3', 'RandomGuy1', 'RandomGuy2', 'RandomGuy3', 'RandomGuy4', 'TitForTat1', 'ResentfulGuy1', 'ResentfulGuy2', 'TrustingGuy1', 'Thanos1', 'Thanos2', 'MidResentful1', 'MidResentful2', 'ReverseTft1', 'ReverseTft2']
('NiceGuy', 286.0)
('BadGuy', 505.3333333333333)
('MainlyNice', 349.25)
('MainlyBad', 404.3333333333333)
('RandomGuy', 374.5)
('TitForTat', 386.0)
('ResentfulGuy', 446.5)
('TrustingGuy', 316.0)
('Thanos', 397.0)
('MidResentful', 426.0)
('ReverseTft', 365.0)


In [15]:
import matplotlib.pyplot as plt
%matplotlib inline